In [1]:
import numpy as np
import torch
import glob
import sklearn.preprocessing as skp

from dataset import PPG_Dataset
from models import vgg16_bn
from sklearn.metrics import classification_report
from tqdm import tqdm
from make_args import Args

- args

In [2]:
args = Args('./config/VGG16_OnlyPPG.json')

- 필요 함수

In [3]:
def normalize_data(data, z_score_norm=True, min_max_norm=False):
    norm_data = []
    data = data.data.cpu().numpy()

    for i in range(len(data)):
        target_data = data[i].copy()
        if z_score_norm:
            target_data = (target_data - target_data.mean()) / (target_data.std() + 1e-17)
        if min_max_norm:
            target_data = skp.minmax_scale(target_data, (-1, 1), axis=1)
        norm_data.append(target_data)

    return torch.from_numpy(np.array(norm_data)).type(torch.FloatTensor)

- Define Device

In [4]:
if torch.cuda.is_available():
    DEVICE = torch.device('cuda')
else:
    DEVICE = torch.device('cpu')

print("Using Pytorch Versions:", torch.__version__, ' Device:', DEVICE)

Using Pytorch Versions: 2.1.1+cu118  Device: cuda


- load partition

In [5]:
partition = np.load(args.partition_path, allow_pickle=True).item()

testset = partition['testset']

In [6]:
len(testset)

165

- dataloader

In [7]:
batch_size = 33 # testset
num_workers = args.num_workers

In [8]:
dataloader_instance = PPG_Dataset(filepaths=testset, sampling_rate=args.target_sampling_rate, 
                                  min_max_norm=args.min_max_norm, z_score_norm=args.z_score_norm, interp=args.interp_method)
dataloader = torch.utils.data.DataLoader(dataloader_instance,
                                             batch_size = batch_size,
                                             shuffle = None,
                                             num_workers = num_workers,
                                             drop_last = True,
                                             pin_memory = True)

- load model

In [9]:
cls_weight_path = args.model_save_path

cls_model = vgg16_bn(in_channels=args.in_channels, num_classes=args.num_classes)
cls_weights = torch.load(cls_weight_path)
cls_model.load_state_dict(cls_weights)
cls_model.to(DEVICE)
cls_model.eval()

VGG(
  (features): Sequential(
    (0): Conv1d(1, 32, kernel_size=(3,), stride=(1,), padding=(1,))
    (1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv1d(32, 32, kernel_size=(3,), stride=(1,), padding=(1,))
    (4): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool1d(kernel_size=3, stride=3, padding=0, dilation=1, ceil_mode=False)
    (7): Conv1d(32, 64, kernel_size=(3,), stride=(1,), padding=(1,))
    (8): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace=True)
    (10): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,))
    (11): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU(inplace=True)
    (13): MaxPool1d(kernel_size=3, stride=3, padding=0, dilation=1, ceil_mode=False)
    (14): Conv1d(64, 128, kernel_size=(3,), stride=(1,

- inference

In [10]:
z_score_norm = args.z_score_norm_rescale
min_max_norm = args.min_max_norm_rescale

In [11]:
y_trues_list = []
y_preds_list = []

for data in tqdm(dataloader, total=len(dataloader)):    
    with torch.no_grad():
        # data extraction
        ppg, label = data
        ppg = ppg.to(DEVICE)
        
        # normalize
        if z_score_norm == True and min_max_norm == False:
            ppg = normalize_data(ppg, z_score_norm=True, min_max_norm=False)

        elif z_score_norm == False and min_max_norm == True:
            ppg = normalize_data(ppg, z_score_norm=False, min_max_norm=True)

        elif z_score_norm == True and min_max_norm == True:
            ppg = normalize_data(ppg, z_score_norm=True, min_max_norm=True)

        elif z_score_norm == False and min_max_norm == False:
            ppg = normalize_data(ppg, z_score_norm=False, min_max_norm=False)

        input_data = ppg.to(DEVICE)
        
        y_pred = cls_model(input_data).data.cpu().numpy()
        y_pred_max = list(np.argmax(y_pred, -1))
        label = list(label.data.cpu().numpy())
        
        y_trues_list.extend(label)
        y_preds_list.extend(y_pred_max)

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:04<00:00,  1.16it/s]


- score

In [12]:
print(classification_report(y_trues_list, y_preds_list, digits=4))

              precision    recall  f1-score   support

           0     0.9675    0.9225    0.9444       129
           1     0.7619    0.8889    0.8205        36

    accuracy                         0.9152       165
   macro avg     0.8647    0.9057    0.8825       165
weighted avg     0.9226    0.9152    0.9174       165

